In [ ]:
# Download Dependencies

!pip install git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/CLIP
!pip install git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/ml-mobileclip
!pip install git+https://github.com/THU-MIG/yoloe.git#subdirectory=third_party/lvis-api
!pip install git+https://github.com/THU-MIG/yoloe.git

!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_blt.pt

from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="jameslahm/yoloe", filename="yoloe-11s-seg.pt", local_dir='.')

In [ ]:
# Import the YOLOE model from the ultralytics library
from ultralytics import YOLOE

# Load the YOLOE model
model = YOLOE("yoloe-11s-seg.pt")

# Define the class names that the model will recognize
names = ["crashed car"]
model.set_classes(names, model.get_text_pe(names))


In [ ]:
# Import necessary libraries
import os
import torch
from PIL import Image

# Convert bounding box from xywhn format (normalized center coordinates) to xyxy format (absolute coordinates)
def xywhn_to_xyxy(bbox, img_width, img_height):
    x_c, y_c, w, h = bbox
    x1 = (x_c - w / 2) * img_width
    y1 = (y_c - h / 2) * img_height
    x2 = (x_c + w / 2) * img_width
    y2 = (y_c + h / 2) * img_height
    return [x1, y1, x2, y2]

# Compute IoU between two bounding boxes
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = max(0, boxA[2] - boxA[0]) * max(0, boxA[3] - boxA[1])
    boxBArea = max(0, boxB[2] - boxB[0]) * max(0, boxB[3] - boxB[1])

    iou = interArea / (boxAArea + boxBArea - interArea + 1e-6)  # Small epsilon to avoid division by zero
    return iou


dataset_folder = "/content/dataset"
annotations_folder = "/content/dataset/annotations"

IOU_THRESHOLD = 0.5
total_pred = 0  # Count of all predicted boxes
all_ious = []   # List to store IoUs for valid matches

# Iterate through the images in the dataset folder
for filename in os.listdir(dataset_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(dataset_folder, filename)  # Full path to the image file
        image = Image.open(image_path)
        width, height = image.size

        # Make predictions on the image using the model (set confidence threshold to 0.26)
        results = model.predict(image_path, conf=0.26)
        pred_boxes = results[0].boxes.xywhn.cpu().numpy()  # Predicted bounding boxes (xywhn format)
        results[0].show()

        # Load the ground truth annotations for the current image
        annotation_path = os.path.join(annotations_folder, os.path.splitext(filename)[0] + ".txt")
        if not os.path.exists(annotation_path):
            print(f"No annotation for {filename}")
            continue

        with open(annotation_path, "r") as f:
            lines = f.readlines()
        gt_boxes = [list(map(float, line.strip().split()[1:])) for line in lines]  # Extract ground truth bounding boxes

        # Convert both predicted and ground truth boxes to xyxy format
        pred_xyxy = [xywhn_to_xyxy(box, width, height) for box in pred_boxes]
        total_pred += len(pred_xyxy)
        gt_xyxy = [xywhn_to_xyxy(box, width, height) for box in gt_boxes]

        matched_gt = set()  # Set to keep track of matched ground truth boxes

        # Compare predicted boxes with ground truth boxes
        for i, pbox in enumerate(pred_xyxy):
            best_iou = 0
            best_j = -1
            for j, gbox in enumerate(gt_xyxy):
                if j in matched_gt:
                    continue
                iou = compute_iou(pbox, gbox)
                if iou > best_iou:
                    best_iou = iou
                    best_j = j

            # Check if the best IoU is above the threshold
            if best_iou >= IOU_THRESHOLD:
                all_ious.append(best_iou)
                matched_gt.add(best_j)
                print(f"{filename} - Match: Pred {i} ↔ GT {best_j} | IoU: {best_iou:.4f}")
            else:
                print(f"{filename} - Pred {i} → No match (max IoU = {best_iou:.4f})")

# Calculate and print the mean IoU across all matches
if all_ious:
    mean_iou = sum(all_ious) / total_pred
    print(f"\n📊 Mean IoU over all matches: {mean_iou:.4f} (with {total_pred} predictions and {len(all_ious)} matches)")
else:
    print("\n❌ No valid bounding box matches found.")


In [ ]:
# Use the model without IoU calculation

import os
from PIL import Image

dataset_folder = "/content/dataset"
for filename in os.listdir(dataset_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(dataset_folder, filename)
            results = model.predict(image_path)
            results[0].show()

In [ ]:
# Renove dataset folder
!rm -rf /content/dataset